In [7]:
import datetime
import fetcher
import compute
import pandas as pd
from finta import TA
import numpy as np

startDate = datetime.date(2020, 1, 1)
endDate = datetime.date(2024, 8, 10)
#endDate = datetime.date.today()
Stocks_list = []#"NVDA", "AAPL", "DKNG", "UBER"

# Retrieving tickers from a file
stocks = pd.read_csv("SCANS\\US_2024-08-10.csv")
#Neglecting tickers with Exchange values "AMEX" and "OTC"
stocks = stocks[~stocks["Exchange"].isin(["OTC", "AMEX"])]
#stocks = stocks[~stocks["Industry"].isin(["Biotechnology", "Pharmaceuticals: major", "Medical Specialties"])]
for ticker in stocks["Symbol"].tolist()[1:]:
    Stocks_list.append(ticker)

df_index = fetcher.__download_data( ["SPY"], startDate, endDate, "US")

leng = 126

focus = pd.DataFrame(columns=['ticker','ADV'])
for ticker in Stocks_list:
    
    #Neglecting tickers with .A or .B or "/" in their names
    if len(ticker.split(".")) >1 or len(ticker.split("/")) >1:
        continue
    df = fetcher.__download_data([ticker], startDate, endDate, "US")

    if len(df) < leng:
        continue
    
        
    else:
        # Avg Volume*Price
        ADV = compute.avg_dollar_vol(df, 30)
        new_row = {"ticker":ticker, "ADV": ADV}
        focus = focus.append(new_row, ignore_index=True)

   



### Filtering only based on ADV for calculation if weighted_sum_close of each Industry

In [8]:
final_focus = focus.loc[ (focus['ADV'] >=10000000) ]
len(final_focus["ticker"].values)

2088

### Sorting by ADV

In [9]:
final_focus = pd.DataFrame(final_focus.sort_values(by=['ADV'], ascending=False).to_numpy(), 
                   index=final_focus.index, columns=final_focus.columns)

### Grouping by Sectors & Industries

In [10]:
sectors = {}
for index, row in final_focus.iterrows():
    exchange = stocks.loc[stocks['Symbol'] == row["ticker"]]["Exchange"].values[0]
    sector = stocks.loc[stocks['Symbol'] == row["ticker"]]["Sector"].values[0]
    #print(sector)
    industry = stocks.loc[stocks['Symbol'] == row["ticker"]]["Industry"].values[0]
    #print(industry)
    if sector not in sectors.keys():
        sectors[sector] = {}
        
    if industry not in sectors[sector].keys():
        #print(industry)
        sectors[sector][industry] = []
    
    #print(sectors)
            #print(sectors[sector])
    sectors[sector][industry].append(exchange+":"+row["ticker"])

### Plotting RS line of Industries w.r.t realtive stock and comparing with the relative stock

In [11]:
import plotly.graph_objects as go
import numpy as np

industry_index = {}
for i in range(len(sectors.values())):
    for key, values in list(sectors.values())[i].items():
        if len(values)>=20:
            stocks_list = values[0:20]
        else:
            stocks_list = values
        df_index = fetcher.__download_data( ["SPY"], startDate, endDate, "US")
        df_index = df_index[-126:]

        total_ADV = 0
        total_MCap = 0
        weighted_sum_close_ADV = 0
        weighted_sum_close_MCap = 0
        for ticker in stocks_list:
            #ticker = ticker.split(":")[1]
            #Neglecting tickers with .A or .B or "/" in their names
            if len(ticker.split(".")) >1 or len(ticker.split("/")) >1:
                continue
            df = fetcher.__download_data([ticker], startDate, endDate, "US")
            df = df[-126:]
            market_cap = stocks.loc[stocks['Symbol'] == ticker]["Market capitalization"].values[0]
            ADV = focus.loc[focus['ticker'] == ticker]["ADV"].values[0]
            total_ADV += ADV 
            total_MCap += market_cap
            weighted_sum_close_ADV += ADV*df["Close"]#[-252:0]
            weighted_sum_close_MCap += market_cap*df["Close"]
        weighted_sum_close_ADV /= total_ADV
        RS_ADV = weighted_sum_close_ADV.values/df_index["Close"]
        weighted_sum_close_MCap /= total_MCap
        RS_MCap = weighted_sum_close_MCap.values/df_index["Close"]

        #Saving the index of each industry
        industry_index[key]=weighted_sum_close_ADV




        #################################################################################
        #Normalization  ( To directly compare slopes of two lines slope1_normalized and slope2 since both are in same scale)
        lower = min(RS_ADV)
        upper = max(RS_ADV)

        _min = min(df_index["Close"])
        _max = max(df_index["Close"])
        df_index_normalized = lower + ((df_index["Close"] - _min)/(_max - _min))*(upper-lower)
        x = list(range(0, 126)) #x = df_index["Formatted Date"] #list(range(1, 126))
        y1_normalized = df_index_normalized

        # Select the last 20 data points for regression
        x_last_20 = x[-20:]
        y1_normalized_last_20 = y1_normalized[-20:]

        # Perform linear regression for the last 20 points
        slope1_normalized, intercept1_normalized = np.polyfit(x_last_20, y1_normalized_last_20, 1)
        
        #######################################################################################




        x = list(range(0, 126)) #x = df_index["Formatted Date"] #list(range(1, 126))
        y1 = df_index["Close"]     # First line (e.g., stock price 1)
        y2 = RS_ADV    # Second line (e.g., stock price 2)

        # Select the last 20 data points for regression
        x_last_20 = x[-20:]
        y1_last_20 = y1[-20:]
        y2_last_20 = y2[-20:]

        # Perform linear regression for the last 20 points
        slope1, intercept1 = np.polyfit(x_last_20, y1_last_20, 1)
        slope2, intercept2 = np.polyfit(x_last_20, y2_last_20, 1)

        
        # Calculate the regression lines
        y1_regression = [slope1 * xi + intercept1 for xi in x_last_20]
        y2_regression = [slope2 * xi + intercept2 for xi in x_last_20]

        # Create the figure
        fig = go.Figure()
        
        # Add the first line (left y-axis)
        fig.add_trace(go.Scatter(
            x=x, 
            y=y1, 
            name="Relative Stock close",
            yaxis="y1",  # Bind to y-axis 1 (left)
            line=dict(color='blue')
        ))

        # Add the second line (right y-axis)
        fig.add_trace(go.Scatter(
            x=x, 
            y=y2, 
            name="RS",
            yaxis="y2",  # Bind to y-axis 2 (right)
            line=dict(color='red')
        ))

        # Add the linear regression line for y1 (left y-axis)
        fig.add_trace(go.Scatter(
            x=x_last_20, 
            y=y1_regression, 
            name="Line 1 Regression",
            yaxis="y1",  # Bind to y-axis 1 (left)
            line=dict(color='blue', dash='dash')
        ))

        # Add the linear regression line for y2 (right y-axis)
        fig.add_trace(go.Scatter(
            x=x_last_20, 
            y=y2_regression, 
            name="Line 2 Regression",
            yaxis="y2",  # Bind to y-axis 2 (right)
            line=dict(color='red', dash='dash')
        ))

        # Update layout to include both y-axes
        fig.update_layout(
            title=key,
            xaxis=dict(
                title="X-axis",
                domain=[0.1, 0.9]  # Adjusting the domain to give space for both y-axes
            ),
            yaxis=dict(
                title="Relative Stock close",
                titlefont=dict(color="blue"),
                tickfont=dict(color="blue")
            ),
            yaxis2=dict(
                title="RS",
                titlefont=dict(color="red"),
                tickfont=dict(color="red"),
                anchor="x",  # Bind this y-axis to the same x-axis
                overlaying="y",  # Overlay it on the first y-axis
                side="right"  # Position it on the right
            ),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="center",
                x=0.5
            ),
            hovermode="x unified",
            dragmode="pan",  # Enable panning with left click and drag
        )

        # Enable scroll zoom
        fig.update_layout(
            autosize=True,
        )

        # Activate scroll zooming and disable box zoom
        config = dict(
            scrollZoom=True,  # Enable zooming with mouse scroll
            displayModeBar=True,  # Show the mode bar for interactivity options
            modeBarButtonsToRemove=['zoom2d'],  # Remove the zoom box tool
        )

        # Show the figure
        fig.show(config=config)

        #################################################################################################################################
        """# Example data
        x = list(range(0, 126)) #x = df_index["Formatted Date"] #list(range(1, 126))
        y1 = df_index["Close"]     # First line (e.g., stock price 1)
        y2 = RS_MCap    # Second line (e.g., stock price 2)

        # Select the last 20 data points for regression
        x_last_20 = x[-20:]
        y1_last_20 = y1[-20:]
        y2_last_20 = y2[-20:]

        # Perform linear regression for the last 20 points
        slope1, intercept1 = np.polyfit(x_last_20, y1_last_20, 1)
        slope2, intercept2 = np.polyfit(x_last_20, y2_last_20, 1)

        # Calculate the regression lines
        y1_regression = [slope1 * xi + intercept1 for xi in x_last_20]
        y2_regression = [slope2 * xi + intercept2 for xi in x_last_20]

        # Create the figure
        fig = go.Figure()
        
        # Add the first line (left y-axis)
        fig.add_trace(go.Scatter(
            x=x, 
            y=y1, 
            name="Relative Stock close",
            yaxis="y1",  # Bind to y-axis 1 (left)
            line=dict(color='blue')
        ))

        # Add the second line (right y-axis)
        fig.add_trace(go.Scatter(
            x=x, 
            y=y2, 
            name="RS",
            yaxis="y2",  # Bind to y-axis 2 (right)
            line=dict(color='red')
        ))

        # Add the linear regression line for y1 (left y-axis)
        fig.add_trace(go.Scatter(
            x=x_last_20, 
            y=y1_regression, 
            name="Line 1 Regression",
            yaxis="y1",  # Bind to y-axis 1 (left)
            line=dict(color='blue', dash='dash')
        ))

        # Add the linear regression line for y2 (right y-axis)
        fig.add_trace(go.Scatter(
            x=x_last_20, 
            y=y2_regression, 
            name="Line 2 Regression",
            yaxis="y2",  # Bind to y-axis 2 (right)
            line=dict(color='red', dash='dash')
        ))

        # Update layout to include both y-axes
        fig.update_layout(
            title=key,
            xaxis=dict(
                title="X-axis",
                domain=[0.1, 0.9]  # Adjusting the domain to give space for both y-axes
            ),
            yaxis=dict(
                title="Relative Stock close",
                titlefont=dict(color="blue"),
                tickfont=dict(color="blue")
            ),
            yaxis2=dict(
                title="RS",
                titlefont=dict(color="red"),
                tickfont=dict(color="red"),
                anchor="x",  # Bind this y-axis to the same x-axis
                overlaying="y",  # Overlay it on the first y-axis
                side="right"  # Position it on the right
            ),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="center",
                x=0.5
            ),
            hovermode="x unified",
            dragmode="pan",  # Enable panning with left click and drag
        )

        # Enable scroll zoom
        fig.update_layout(
            autosize=True,
        )

        # Activate scroll zooming and disable box zoom
        config = dict(
            scrollZoom=True,  # Enable zooming with mouse scroll
            displayModeBar=True,  # Show the mode bar for interactivity options
            modeBarButtonsToRemove=['zoom2d'],  # Remove the zoom box tool
        )

        # Show the figure
        fig.show(config=config)"""